In [ ]:
import os
os.chdir("../")
os.getcwd()

In [ ]:
from attacker.query import *
from attacker.utils import *
from victim import *
from victim.interface import  fetch_victim_model

set_seed(42)

# Query victim model & Train attacker model

In [ ]:
# logits
k = 3

# get dataset in dataloader
trainloader, testloader, outputs = get_dataloader(config["victim"]["data"])

# query & save test data with labels
querytestloader = query_victim(config["victim"], outputs, testloader, len(testloader.dataset), train=False)

# sample train data on querytype.size
sampledtrainloader = query_type(config["victim"], outputs, trainloader, config["query_size"],
                                f'queried/query_traindata_{config["victim"]["data"]}_{config["victim"]["model_name"]}_k{k}',
                                config["query_type"])

# fetch victim
victim_model = fetch_victim_model(args=config["victim"])

# initialize attacker model
attacker = get_model(config["attacker"], outputs)

# train attacker model
attacker_result = attacker_training_w_logits(victim_model, attacker, sampledtrainloader, querytestloader, k)

# save & visualize model inference
title = f'Trial-A_{config["attacker"]}_{config["victim"]["model_name"]}_{config["victim"]["data"]}{config["query_type"]}_{config["query_size"]}'
percent = save_visualize(attacker, attacker_result,title )
results_dict={"Victim":config["victim"]["model_name"],
                "Dataset":config["victim"]["data"],
                "QueryType": config["query_type"],
                "QuerySize": config["query_size"],
                "Queried Output": k ,
                "Attacker": config["attacker"],
                "Train Loss": percent[0],
                "Train Accuracy": percent[1],
                "Test Loss": percent[2],
                "Test Accuracy": percent[3]}
print(results_dict)

# Parameters

In [ ]:
parameters = {
        "query_size": [20000],
        "query_type": ['coreset'],
        "victim":[{ "data": CIFAR_10, "model_name": RESNET50}],
        "attacker":[RESNET34],
        "k-logits": [3, 10]
}

# Special Investigations

K-Logits

In [ ]:
print(f'config={config}\n\nparameters={parameters}\n')
Results = []
# Iterate through Victim Model & Dataset
for victim in parameters["victim"]:
    print('---------------------------------------------------------------------------')
    # get dataset in dataloader
    trainloader, testloader, outputs = get_dataloader(victim["data"])

    # query test data
    querytestloader = query_victim(victim, outputs, testloader, len(testloader.dataset), train=False)

    # Iterate through Attacker Model
    for attacker_type in parameters["attacker"]:
        # Iterate Through Query Type
        for querytype in parameters["query_type"]:
            # Iterate Through Query Size
            for size in parameters["query_size"]:
                # Iterate Through Logits size
                for k in parameters["k-logits"]:
                    print('-----------------------------------------------------------------------------')
                    print(f'-----------------------Dataset: {victim["data"]}----------------------------')
                    print(f'--------Victim: {victim["model_name"]} Attacker: {attacker_type}------------')
                    print(f'---------------Query Type: {querytype} Query Size: {size}-------------------')

                    # sample train data on querytype.size
                    sampledtrainloader = query_type(victim, outputs, trainloader, size,
                                    f'queried/query_traindata_{victim["data"]}_{victim["model_name"]}_k{k}', querytype)

                    # fetch victim
                    victim_model = fetch_victim_model(args=victim)

                    # initialize attacker model
                    attacker = get_model(attacker_type, outputs)

                    # train attacker model
                    attacker_result = attacker_training_w_logits(victim_model, attacker, sampledtrainloader, querytestloader, k)

                    # save & visualize model inference
                    title = f'A_{attacker_type}_{victim["model_name"]}_{victim["data"]}_{querytype}_{size}'
                    percent = save_visualize(attacker, attacker_result,title )
                    results_dict={"Victim":victim["model_name"],
                                    "Dataset":victim["data"],
                                    "QueryType": querytype,
                                    "QuerySize": size,
                                    "Queried Output": k,
                                    "Attacker": attacker_type,
                                    "Train Loss": percent[0],
                                    "Train Accuracy": percent[1],
                                    "Test Loss": percent[2],
                                    "Test Accuracy": percent[3]}

                    print(results_dict)
                    Results.append(results_dict)
                    print('-----------------------------------------------------------------------------')

In [ ]:
Results